<a href="https://colab.research.google.com/github/dfklsna/sangwoo-Kim/blob/main/car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


targetUrl = "https://raw.githubusercontent.com/dfklsna/sangwoo-Kim/main/auto-mpg.csv"
df = pd.read_csv(targetUrl, sep=';')

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 1 columns):
 #   Column                                                                                Non-Null Count  Dtype 
---  ------                                                                                --------------  ----- 
 0   mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name  398 non-null    object
dtypes: object(1)
memory usage: 3.2+ KB


In [69]:
df[['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'car name']] = df['mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name'].str.split(',', expand=True)

df.drop('mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   mpg           398 non-null    object
 1   cylinders     398 non-null    object
 2   displacement  398 non-null    object
 3   horsepower    398 non-null    object
 4   weight        398 non-null    object
 5   acceleration  398 non-null    object
 6   model year    398 non-null    object
 7   origin        398 non-null    object
 8   car name      398 non-null    object
dtypes: object(9)
memory usage: 28.1+ KB


In [70]:
df['origin'] = df['origin'].replace({1: '미국차', 2: '유럽차', 3: '일본차'})

In [71]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

df = df.replace('?', np.nan)

df = df.drop('car name', axis=1)

df = df.dropna()

X = df.drop('mpg', axis=1)
y = df['mpg']

neg_mse_scores = cross_val_score(LinearRegression(), X.fillna(X.mean(numeric_only=True)).values, y, scoring='neg_mean_squared_error', cv=5)

mse_scores = -neg_mse_scores
mean_mse = mse_scores.mean()

print("Negative MSE Scores:", neg_mse_scores)
print("Mean MSE Score:", mean_mse)

Negative MSE Scores: [-14.97430765 -10.90595243  -5.99170861 -15.58754466 -27.84474308]
Mean MSE Score: 15.060851285619872


In [72]:
from sklearn.model_selection import cross_val_score

lr = LinearRegression()

neg_mse_scores = cross_val_score(lr, X, y, scoring="neg_mean_squared_error", cv=5)

mse_scores = -neg_mse_scores

mean_mse = mse_scores.mean()

print("Negative MSE Scores:", neg_mse_scores)
print("Mean MSE Score:", mean_mse)

Negative MSE Scores: [-14.97430765 -10.90595243  -5.99170861 -15.58754466 -27.84474308]
Mean MSE Score: 15.060851285619856


In [77]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pipeline = make_pipeline(PolynomialFeatures(degree=3), Ridge())

param_grid = {'ridge__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}

grid_search = GridSearchCV(pipeline, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_scaled, y)

best_alpha = grid_search.best_params_['ridge__alpha']
best_score = -grid_search.best_score_

print("Best Alpha:", best_alpha)
print("Best Score (neg_mean_squared_error):", best_score)

Best Alpha: 10
Best Score (neg_mean_squared_error): 10.055977603736377
